In [1]:
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from src.data.dataloader import MURADataset
from src.models.models import CNN, CNN_3
from netcal.metrics import ECE
from src.models.utils import pred, get_variable
from torch.autograd import Variable
from torch.nn import functional as F
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# input image
#image_file = 'data/MURA-v1.1/valid/XR_ELBOW/patient11204/study1_negative/image1.png'
#image_file = 'data/MURA-v1.1/train/XR_FINGER/patient04887/study1_negative/image2.png'
#image_file = 'data/MURA-v1.1/valid/XR_HAND/patient11212/study1_negative/image2.png'
image_file = 'data/MURA-v1.1/valid/XR_SHOULDER/patient11723/study1_positive/image3.png'

#model_path = 'models/STATEtrained_model_epocs2_24-03-2022_14.pt'
#model_path = 'models/STATEtrained_model_epocs70_24-03-2022_22.pt'

# The model that we want to use
#model_path = 'models/STATEtrained_model_epocs100_14_04_18_trans_1.pt'

# Model on only two epochs:
#model_path = 'models/STATEtrained_model_epocs2_15_04_15_trans_0.pt'
model_path = 'models/STATEtrained_model_epocs2_15_04_20_trans_0.pt'

device = "cuda" if torch.cuda.is_available() else "cpu"

use_cuda = torch.cuda.is_available()


net = CNN(input_channels=3, input_height=256, input_width=256, num_classes=7).to(
    device
)

last_conv_name = 'conv5'

net.load_state_dict(
    torch.load(
        model_path,
        map_location=torch.device(device),
    )
)

net.eval()

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.2, inplace=False)
  (avgpool): AvgPool2d(kernel_size=32, stride=32, padding=0)
  (l_out): Linear(in_features=64, out_features=7, bias=True)

In [9]:
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get(last_conv_name).register_forward_hook(hook_feature)

# get the softmax weight
params = list(net.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

In [10]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam


preprocess = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# load test image
img_pil = Image.open(image_file).convert("RGB")
img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = net(img_variable)

# load the imagenet category list
classes = [0,1,2,3,4,5,6]


h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

# generate class activation mapping for the top1 prediction
CAMs = returnCAM((features_blobs[0]), weight_softmax, [idx[0]])

# render the CAM and output
print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
img = cv2.imread(image_file)
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('CAM.jpg', result)



0.659 -> 0
0.092 -> 4
0.058 -> 1
0.056 -> 3
0.054 -> 2
output CAM.jpg for the top1 prediction: 0


True